# In-Context Learning
* Be careful when doing this, as the cost is very high.

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.45


In [3]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [4]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import ICLClassifier

In [5]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_train.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_train = df.drop('target', axis=1)
y_train = df['target']
print(len(x_train))

300


In [6]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4')

params = {'columns_name': True}

iclm = ICLClassifier(llm_model = llm_model)

### Make prompt

In [7]:
icl_prompt = iclm.fit(x_train, y_train)

8163


In [8]:
print(icl_prompt)


        Output the values according to all of the following conditions.
        ・The output should be numeric only.
        ・Do not output any text.
        ・Non-numeric output will result in an error.
        ・Predict the probability value as accurately as possible. Please be as detailed as possible.
        ・The rightmost column with a value of 0 or 1 is 'target'.
        ------------------
        1.14,-1.32,-1.92,1.8,1.0
0.29,-0.55,-0.63,0.33,1.0
0.33,-0.8,-0.84,0.25,1.0
0.13,1.03,0.59,0.96,1.0
0.01,1.18,0.79,0.77,1.0
0.88,-1.15,-1.57,1.3,1.0
0.55,-0.69,-0.96,0.85,1.0
0.44,0.3,-0.19,1.2,1.0
-0.19,2.04,1.57,0.84,1.0
0.23,0.61,0.21,0.93,1.0
-0.15,1.88,1.42,0.83,1.0
0.77,-1.03,-1.39,1.11,1.0
0.84,-1.32,-1.65,1.1,1.0
0.37,-0.98,-1.0,0.23,1.0
0.32,0.2,-0.14,0.86,1.0
0.38,1.09,0.41,1.56,1.0
0.28,0.48,0.07,0.95,1.0
0.15,0.86,0.46,0.9,1.0
0.97,-1.13,-1.64,1.52,1.0
0.83,-1.28,-1.61,1.11,1.0
0.13,1.07,0.62,0.98,1.0
0.09,0.95,0.57,0.81,1.0
0.69,-0.72,-1.11,1.13,1.0
0.96,-1.43,-1.83,1.3,1.0
0

### Prediction

In [9]:
file_name = f'pseudodata_{columns}f_{train_data}_test.csv'


df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

In [10]:
y_proba = iclm.predict(x_test)

y_pred = (y_proba > 0.5).astype(int)

output = pd.DataFrame(y_proba, columns=['pred'])
prefix = 1
output.to_csv(f'../data/icl_output/pred_{prefix}.csv')

 67%|██████▋   | 467/700 [23:33<12:26,  3.20s/it]Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-rKmEAn8qrwuz778vK8jNCcB5 on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
 67%|██████▋   | 469/700 [23:43<14:20,  3.72s/it]Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitErro

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [12]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.62
Precision: 0.5697674418604651
Recall: 0.98
F1 score: 0.7205882352941176
ROC-AUC: 0.5388


In [8]:
columns = 4
train_data = 300

file_name = f'pseudodata_{columns}f_{train_data}_test.csv'

df = pd.read_csv(f'../data/pseudodata/{file_name}')
x_test = df.drop('target', axis=1)
y_test = df['target']

prefix = 1
y_proba = pd.read_csv(f'../data/icl_output/pred.csv')
y_proba = y_proba.drop('Unnamed: 0', axis=1)
y_pred = (y_proba > 0.5).astype(int)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

ValueError: Found input variables with inconsistent numbers of samples: [700, 300]